In [ ]:
# import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix

import datetime
plt.rcParams["figure.figsize"] = (20,10)

In [ ]:
# settings
shift = 15
window = 45
model_type = "autoreg15" # for autoreg - autoreg{shift]; for multiNN - PC, AsyH, ap_index, BzGSE, all_parameters
model_type2 = "autoreg" # for autoreg - autoreg; for multiNN - PC, AsyH, ap_index, BzGSE, all_parameters
best_model = True # load best model (true) or model from epoch (false)
epoch = "03"
mae = "0.0561"

In [ ]:
# data loading
size=1 # select subset (>1) or whole test_data (1)
param_t = np.load(f"../data/{model_type2}/shift-{shift}-windows-{window}/test_index_timestamp.npy", allow_pickle = True).flatten()
param_ytest = np.load(f"../data/{model_type2}/shift-{shift}-windows-{window}/y_test.npy", allow_pickle = True).flatten()
param_bin = np.load(f"../data/{model_type2}/shift-{shift}-windows-{window}/test_index_bin.npy", allow_pickle = True).flatten()
if best_model:
    param_pred = np.load(f"../prediction/y_pred-{model_type2}-shift-{shift}-windows-{window}.npy, allow_pickle = True").flatten()
else:
    param_pred = np.load(f"../prediction/{model_type2}/shift-{shift}-windows-{window}/y_pred_epoch-{epoch}-mae-{mae}.npy", allow_pickle = True).flatten()

# create dataframe
param_df = pd.DataFrame(data={f"{model_type}_test": param_ytest[:int(len(param_ytest)/size)], f"{model_type}_pred": param_pred[:int(len(param_pred)/size)], f"{model_type}_bin":param_bin[:int(len(param_ytest)/size)], f"{model_type}_time":param_t[:int(len(param_ytest)/size)]})
param_df[f"{model_type}_bin"] = param_df[f"{model_type}_bin"].str[:5]

param_df["index_time_bin"]=param_df[f"{model_type}_time"]+" "+param_df[f"{model_type}_bin"]
param_df=param_df.set_index("index_time_bin")

#param_df.to_csv(f"{model_type}_predictions_on_test-data_with_time-bin.csv")


In [ ]:
len(param_pred)

In [ ]:
# load data (if you want load from .csv)
# data = pd.read_csv(f'{model_type}_predictions_on_test-data_with_time-bin.csv')

# or select data from param_df
data = param_df

data[f"{model_type}_time"] = pd.to_datetime(data[f"{model_type}_time"])
data

### Coefficient of variation

In [ ]:
global_mean = data[f"{model_type}_test"].mean()
global_mean

In [ ]:
n = len(data)
pred_std=[]
test_std=[]
window_size_cov=[]

# create windows for CoV calculation
for i in range(n):
    subset = data[max(0,i-22):min(i+22+1, n)]
    subset = subset[subset[f"{model_type}_bin"]==data[f"{model_type}_bin"][i]]
    subset = subset[subset[f"{model_type}_time"] <= data[f"{model_type}_time"][i] + datetime.timedelta(minutes=22)]
    subset = subset[subset[f"{model_type}_time"] >= data[f"{model_type}_time"][i] - datetime.timedelta(minutes=22)]
    pred_std.append(subset[f"{model_type}_pred"].std())
    test_std.append(subset[f"{model_type}_test"].std())
    window_size_cov.append(len(subset))
    if i%10000 == 0: # control the speed of calculations
        print(i)

In [ ]:
data[f"{model_type}_std_pred"] = pred_std
data[f"{model_type}_std_test"] = test_std

# calculate CoV
data[f"{model_type}_cov_pred"] = data[f"{model_type}_std_pred"]/global_mean
data[f"{model_type}_cov_test"] = data[f"{model_type}_std_test"]/global_mean

# if you want to save size of CoV windows
# data[f"{model_type}_count_for_cov"] = window_size_cov

In [ ]:
# save data
data.to_csv(f'{model_type2}/{model_type}_predictions_on_test-data_with_time-bin_std-cov_full-test.csv')

In [ ]:
data